In [1]:
from main import *

In [14]:
t = pd.read_csv('processed_data/msci_us/historical.csv')

In [15]:
t['ticker'].nunique()

577

In [16]:
t.value_counts('date').sort_index().plot()

In [17]:
t['date'] = pd.to_datetime(t['date']).dt.date

In [18]:
t['exists'] = 1

In [19]:
tl = pd.merge(
	pd.DataFrame({'date': t['date'].unique()}),
	pd.DataFrame({'ticker': t['ticker'].unique()}),
	how='cross'
)
tl = pd.merge(tl, t, on=['date', 'ticker'], how='left')
tl['exists'] = tl['exists'].fillna(0)
tl = tl.sort_values(['date', 'ticker']).reset_index(drop=True)

# add columns for whether a ticker was added or dropped on a given day
tl['added_today'] = (tl['exists'] == 1) & (tl.groupby('ticker')['exists'].shift(1) == 0)
tl['dropped_today'] = (tl['exists'] == 0) & (tl.groupby('ticker')['exists'].shift(1) == 1)

In [20]:
tl['added_today'] = (tl['exists'] == 1) & (tl.groupby('ticker')['exists'].shift(1) == 0)
tl['dropped_today'] = (tl['exists'] == 0) & (tl.groupby('ticker')['exists'].shift(1) == 1)

In [21]:
qt.view(tl[tl['date'] == dt.date(2025, 5, 30)])

Grid(columns_fit='auto', compress_data=True, css_rules_down=['.number-cell {text-align: left;width: 10;}', '.l…

In [22]:
tl_rebal = tl[tl['date'] == dt.date(2025, 5, 30)]
tl_rebal[tl_rebal['dropped_today']].shape
tl_rebal[tl_rebal['added_today']].shape

(28, 9)

(0, 9)

In [23]:
tl_rebal

,date,ticker,rank,name,sector,weight,exists,added_today,dropped_today
23657,2025-05-30,A,304.00,AGILENT TECHNOLOGIES INC,Health Care,0.18,1.00,False,False
23658,2025-05-30,AAPL,435.00,APPLE INC,Information Technology,0.18,1.00,False,False
23659,2025-05-30,ABBV,99.00,ABBVIE INC,Health Care,0.19,1.00,False,False
23660,2025-05-30,ABNB,367.00,AIRBNB INC CLASS A,Consumer Discretionary,0.18,1.00,False,False
23661,2025-05-30,ABT,246.00,ABBOTT LABORATORIES,Health Care,0.18,1.00,False,False
...,...,...,...,...,...,...,...,...,...
24229,2025-05-30,ZBH,395.00,ZIMMER BIOMET HOLDINGS INC,Health Care,0.18,1.00,False,False
24230,2025-05-30,ZBRA,378.00,ZEBRA TECHNOLOGIES CORP CLASS A,Information Technology,0.18,1.00,False,False
24231,2025-05-30,ZM,359.00,ZOOM COMMUNICATIONS INC CLASS A,Information Technology,0.18,1.00,False,False
24232,2025-05-30,ZS,8.00,ZSCALER INC,Information Technology,0.20,1.00,False,False


In [24]:
tl.to_csv('msci_us_index_weights.csv', index=False)